In [104]:
# Import libraries for data manipulation and ETL pipeline
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from pandarallel import pandarallel
pandarallel.initialize(verbose=0)
from tqdm import tqdm

In [105]:
# Establish API connection
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

cid = '3fda75b7146a4769b207ee44017b3abe'
secret = '2a755cb04a18406b9394dbef2f8069dd'

client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [106]:
# Connect to SQL Server
import sqlalchemy as sql
from sqlalchemy import text
import pyodbc
server = 'LAPTOP-V3754MEK' 
database = 'Spotify'

engine = sql.create_engine('mssql+pyodbc://' + server + '/' + database + '?trusted_connection=yes&driver=ODBC+Driver+13+for+SQL+Server')
conn = engine.connect()
results = engine.execute("SELECT track_uri2 FROM dim_tracks ORDER BY track_uri2")
results = pd.Series(results)
results.head()

0    (spotify:track:0002yNGLtYSYtc0X6ZnFvp)
1    (spotify:track:00039MgrmLoIzSpuYKurn9)
2    (spotify:track:0003Z98F6hUq7XxqSRM87H)
3    (spotify:track:0004ExljAge0P5XWn1LXmW)
4    (spotify:track:0005rgjsSeVLp1cze57jIN)
dtype: object

In [107]:
# Extract tracks from API
output_df = pd.DataFrame()
for row in tqdm(results[670001:680001]):
      output = sp.audio_features(row)
      output_df = output_df.append(output)
output_df.head()

100%|██████████| 10000/10000 [16:04<00:00, 10.36it/s]


,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.500,0.936,1,-7.666,1,0.4250,0.0145,0.000000,0.4920,0.816,161.019,audio_features,2EdXuGy7d76Whv0BuJoe6r,spotify:track:2EdXuGy7d76Whv0BuJoe6r,https://api.spotify.com/v1/tracks/2EdXuGy7d76W...,https://api.spotify.com/v1/audio-analysis/2EdX...,288624,4
0,0.407,0.765,4,-6.175,1,0.0344,0.0169,0.004360,0.5460,0.121,153.040,audio_features,2EDXuUJ28BR45U6Cl6Tisj,spotify:track:2EDXuUJ28BR45U6Cl6Tisj,https://api.spotify.com/v1/tracks/2EDXuUJ28BR4...,https://api.spotify.com/v1/audio-analysis/2EDX...,351240,4
0,0.595,0.306,10,-11.048,1,0.0349,0.9290,0.000098,0.1230,0.556,93.872,audio_features,2EDxZbqgbUY5DW445yFt1V,spotify:track:2EDxZbqgbUY5DW445yFt1V,https://api.spotify.com/v1/tracks/2EDxZbqgbUY5...,https://api.spotify.com/v1/audio-analysis/2EDx...,193348,4
0,0.735,0.611,7,-7.561,1,0.0946,0.0440,0.003930,0.0295,0.568,148.143,audio_features,2EDy9UxnD8Rz8KzeW4KzR5,spotify:track:2EDy9UxnD8Rz8KzeW4KzR5,https://api.spotify.com/v1/tracks/2EDy9UxnD8Rz...,https://api.spotify.com/v1/audio-analysis/2EDy...,159133,4
0,0.529,0.136,7,-16.693,1,0.0637,0.9760,0.670000,0.1370,0.345,120.844,audio_features,2EdyBXaB5myLGsBFtiEtA8,spotify:track:2EdyBXaB5myLGsBFtiEtA8,https://api.spotify.com/v1/tracks/2EdyBXaB5myL...,https://api.spotify.com/v1/audio-analysis/2Edy...,272013,4


In [108]:
# Load data into SQL Server
output_df = output_df[['uri', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms', 'time_signature']]
output_df = output_df.rename(columns={'key':'music_key', 'mode':'modal'})
output_df.to_sql("audio_features2", engine, if_exists='append', index=False)
print(f'{len(output_df)} rows imported to database.')
new_query = engine.execute('SELECT COUNT(*) FROM audio_features2')
new_query = pd.Series(new_query)
print(f'App contains {new_query[0]} songs.')

10000 rows imported to database.
App contains (670001,) songs.
